<a href="https://colab.research.google.com/github/RogerHeederer/FundamentalConceptOfDL/blob/master/3_ConvolutionLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref: 봉수골 개발자 이선비님의 깃허브를 참조하였으며, 학습용으로 진행한 테스트입니다

추가적으로 필요한 설명 및 요약들을 제 나름대로 표현하여 작성하였습니다.

In [11]:
import tensorflow as tf
import pandas as pd

In [12]:
(x, y_target), _ = tf.keras.datasets.mnist.load_data()
print(x.shape, y_target.shape)

(60000, 28, 28) (60000,)


In [13]:
# conv2d 레이어에 넣기 위해서는 2차원 데이터를 3차원 형태의 데이터로 변형시켜줘야 한다.
# 28x28을 28x28x1 형태로 변형
x = x.reshape(60000, 28, 28, 1)
y_target = pd.get_dummies(y_target)
print(x.shape, y_target.shape)

(60000, 28, 28, 1) (60000, 10)


<img src = "https://drive.google.com/uc?export=view&id=139o7Zex0s5-QRUv0FQ47j7CT8r_pvj0O" align=left height="400">



>봉수골 개발자 이선비님 슬라이드 자료

##컨볼루션 네트워크에서 기계가 학습하는 것은 필터이다. 필터들을 뜯어보면 전부 w와 같은 가중치로 이루어져있음

X = tf.keras.layers.Input(shape=[28, 28, 1])

Conv2D(3, kernel_size=5, activation='swish')

Conv2D(6, kernel_size=5, activation='swish')

F1, F2, F3 이렇게 필터 3개가 존재하고, 앞선 레이어가 채널이 1개이니, F1은 (5,5,1) F2도 (5,5,1) F3도 (5,5,1) shape을 가진다.

그럼 필터셋 전체는 [F1,F2,F3] 이니 (3,5,5,1)의 형태임 


1. 필터은 3차원 형태로 된 가중치의 모음

2. 필터 하나는 앞선 레이어의 결과인 "특징맵"(Feature Map) 전체를 본다.

3. 필터 하나가 특징맵 하나를 만든다. 즉 필터 개수 만큼 특징맵을 만든다.
필터의 개수가 3개면, 특징맵도 3개 나옴

## 정리 ##

1. 28x28x1 데이터를 3개의 필터에 넣는다

2. F1 F2 F3 필터가 적용되어 만들어진 M1 M2 M3 특징맵 3개가 나온다

3. 이 특징맵 3개에 3차원의 필터 6개들이 적용되어 특징맵 6개를 만든다.

-- 기계야! 이 이미지들이 0~9까지 중 어느 숫자인지 판단하기 위한 가장 적합한 특징맵 6개를 만들어줘.

-- 기계는 필터에 속해있는 가중치들을 최적화 시키면서 가장 좋은 특징맵을 만들기 위해 학습된다


In [14]:
# 모델 만들기
X = tf.keras.layers.Input(shape=[28, 28, 1])

# 3개의 fiter를 사용하겠고, 각 필터의 사이즌 5. 즉 28x28 데이터 1개마다 3개의 필터 적용. 각개 3개의 필터 결과값(=필터셋)이 나옴
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X) 
H = tf.keras.layers.Conv2D(6, kernel_size=6, activation='swish')(H) # 6개의 filter를 사용하겠고 각 필터의 사이즌 6
H = tf.keras.layers.Flatten()(H) # 픽셀 단위로 한줄로 쭉 펼침.
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 6)         654       
_________________________________________________________________
flatten_1 (Flatten)          (None, 2166)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 84)                182028    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                850       
Total params: 183,610
Trainable params: 183,610
Non-trainable params: 0
_____________________________________________________

28 x 28 데이터에  필터(사이즈5)가 훑으면, 그 데이터는 28 - (사이즈5-1) = 24 픽셀이 된다

다시 24 픽셀에 필터(사이즈 6)이 훑으면, 그 데이터는 24 - (사이즈6-1) = 19 픽셀이 된다.

파라미터가 나오는 과정을 살펴보면,


input값에 커널 사이즈 5인 필터가 일차적으로 적용된다. 커널 사이즈 5인 필터는 5x5 = 25개의 파라미터를 가지고 있음. 근데 이게 3개 적용되니까, 25x3은 75. 여기에 필터 1개당 bias 1개씩 더하면,  75 + 3이라서 78개의 파라미터가 됨

다음으로 커널 사이즈 6인 필터가 3개 채널에 전부 각각 적용되니까 6x6x3 = 108

108개의 파라미터들이 총 6세트 있으니 108x6 = 648개 파라미터. 여기에 필터셋 1개당 bias가 적용되니 648 + 6bias = 654개의 파라미터


In [16]:
model.fit(x, y_target, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 1.0500 - accuracy: 0.8506
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0846 - accuracy: 0.9755
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0475 - accuracy: 0.9857
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0350 - accuracy: 0.9894
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0284 - accuracy: 0.9920
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0216 - accuracy: 0.9933
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0201 - accuracy: 0.9944
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0171 - accuracy: 0.9952
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0154 - accuracy: 0.9960
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.014

In [17]:
# 모델 이용해보기
pred = model.predict(x[0:5])

In [20]:
pred.shape

(5, 10)

In [21]:
pd.DataFrame(pred).round(2)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
y_target[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1
